# Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

### If on runpod / vast.ai / etc, spin up an A6000 or A100 pod using a Stable Diffusion template with Jupyter pre-installed.

## Build Environment

In [ ]:
# If running on Vast.AI, copy the code in this cell into a new notebook. Run it, then launch the `dreambooth_runpod_joepenna.ipynb` notebook from the jupyter interface.
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion

In [ ]:
%cd Dreambooth-Stable-Diffusion

#HOUSEKEEPING
!ln -s /usr/bin/python3.6 /usr/bin/python
!ln -s /usr/bin/pip3 /usr/bin/pip
!mkdir training_samples

#BUILD ENV
%pip install omegaconf
%pip install einops
%pip install pytorch-lightning==1.6.5
%pip install protobuf==3.20.1
%pip install test-tube
%pip install transformers
%pip install kornia
%pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
%pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
%pip install setuptools==59.5.0
%pip install pillow==9.0.1
%pip install torchmetrics==0.6.0
%pip install torchvision
%pip install -e .
%pip install gdown
%pip install pydrive

In [ ]:
## DOWNLOAD SD
%pip install gdown

#Link to a Google Drive ckpt of Stable Diffusion
!gdown https://drive.google.com/uc?id={{FILE ID}}

# Regularization Images

Training teaches your new model both your token **but** re-trains your class simultaneously.

From cursory testing, it does not seem like reg images affect the model too much. However, they do affect your class greatly, which will in turn affect your generations.

You can either generate your images here, or use the repos below to quickly download 1500 images.

In [ ]:
# GENERATE 400 images
!python scripts/stable_txt2img.py --seed 10 --ddim_eta 0.0 --n_samples 1 --n_iter 400 --scale 10.0 --ddim_steps 50  --ckpt model.ckpt \
--prompt "person"

# If you don't want to train against "person", change it to whatever you want above, and on some of the cells below:

In [ ]:
# zip up the files for downloading and reuse.
!apt-get install -y zip
!zip -r all_images.zip outputs/

## Download pre-generated regularization images

In [ ]:
# grab the files here: https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images
!git clone https://github.com/JoePenna/Stable-Diffusion-Regularization-Images

!mkdir outputs
!mkdir outputs/txt2img-samples
!mkdir outputs/txt2img-samples/samples

## RECOMMENDED train with "person":

In [11]:
# 1500 images - ddim @ 50 steps, CFG 7.0
!mv -v Stable-Diffusion-Regularization-Images/person_ddim outputs/txt2img-samples outputs/txt2img-samples/samples

renamed 'Stable-Diffusion-Regularization-Images/person_ddim' -> 'outputs/txt2img-samples/samples/person_ddim'
mv: cannot move 'outputs/txt2img-samples' to a subdirectory of itself, 'outputs/txt2img-samples/samples/txt2img-samples'


### If you'd rather try `man` or `woman`, comment the line above, and uncomment one of the ones below:

In [ ]:
# Use man_euler images - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
# !mv -v Stable-Diffusion-Regularization-Images/man_euler outputs/txt2img-samples

# Use man_unsplash images - pictures from various photographers
# !mv -v Stable-Diffusion-Regularization-Images/man_unsplash outputs/txt2img-samples

# Use woman_dimm images - provided by David Bielejeski - ddim
# !mv -v Stable-Diffusion-Regularization-Images/woman_dimm outputs/txt2img-samples

# Upload your training images
Upload 10-20 images of someone to /workspace/Dreambooth-Stable-Diffusion/training_samples

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body 
*   5-12 close-up on face

In [ ]:
# or upload from your Google Drive
%pip install gdown
!gdown https://drive.google.com/uc?id={fileId}
!mv {name of file} training_samples/{name of file}

## Prep Training Variables

Navigate to:
/workspace/Dreambooth-Stable-Diffusion/ldm/data/personalized.py

Change `demoura` in line 11 to whatever you want your token to be.

(Note: The original repo uses `sks` as a token -- but that has a relatively high correlation with the SKS semi-automatic rifle, which affects low-step generations)

e.g. I changed mine to
```python
training_templates_smallest = [
    'joepenna {}',
]
```

The last line of this file trains for `3000` steps.
/workspace/Dreambooth-Stable-Diffusion/configs/stable-diffusion/v1-finetune_unfrozen.yaml

If training a person or subject, keep an eye on your project's `/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `/images/train/samples_gs-00xxxx` generations.

## Start Training (you should also change the code below)

In [ ]:
# START THE TRAINING
# !python "main.py" \
#  --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
#  -t \
#  --actual_resume "model.ckpt" \
#  --reg_data_root "/home/ubuntu/dreambooth-stable-diffusion/outputs/txt2img-samples/samples" \
#  -n "alex_test" \
#  --gpus 0, \
#  --data_root "/home/ubuntu/dreambooth-stable-diffusion/training_samples" \
#  --class_word "person" # << match this word to the class word from regularization images above

python "main.py" --base configs/stable-diffusion/v1-finetune_unfrozen.yaml -t --actual_resume "model.ckpt" --reg_data_root "/home/ubuntu/dreambooth-stable-diffusion/outputs/txt2img-samples/samples/person_ddim" -n "alex_test" --gpus 0, --data_root "/home/ubuntu/dreambooth-stable-diffusion/training_samples" --class_word "person"

## Pruning (12GB to 2GB)

In [12]:
# This will prune around 10GB from the ckpt file, then move it the "trained_models" folder
!python "scripts/prune-ckpt.py" --ckpt logs/training_samples2022-09-28T06-52-20_alex_test/checkpoints/last.ckpt
!mkdir trained_models
!mv logs/training_samples2022-09-28T06-52-20_alex_test/checkpoints/last-pruned.ckpt /home/ubuntu/dreambooth-stable-diffusion/trained_models/training_samples2022-09-28T06-52-20_alex_test.ckpt

prunin' in path: logs/training_samples2022-09-28T06-52-20_alex_test/checkpoints/last.ckpt
dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])
removing optimizer states for path logs/training_samples2022-09-28T06-52-20_alex_test/checkpoints/last.ckpt
This is global step 3000.
saving pruned checkpoint at: logs/training_samples2022-09-28T06-52-20_alex_test/checkpoints/last-pruned.ckpt
New ckpt size: 2.13 GB. Saved 9.99 GB by removing optimizer states


## Generate Images With Your Trained Model!

In [17]:
!python scripts/stable_txt2img.py \
--ddim_eta 0.6 --n_samples 1 --n_iter 16 \
--scale 10.0 --ddim_steps 200 --ckpt "/home/ubuntu/dreambooth-stable-diffusion/logs/training_samples2022-09-28T06-52-20_alex_test/checkpoints/last.ckpt" \
--prompt "a regal, classy painting of alex as darth vader"

Global seed set to 42
Loading model from /home/ubuntu/dreambooth-stable-diffusion/logs/training_samples2022-09-28T06-52-20_alex_test/checkpoints/last.ckpt
Global Step: 3000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.13.self_attn.k_proj.weight', 'logit_scale', 'vision_model.encoder.layers.4.mlp.fc1.bias', 'vision_model.encoder.layers.14.mlp.fc1.bias', 'vision_model.encoder.layers.23.self_attn.v_proj.weight', 'vision_model.encoder.layers.18.self_attn.v_proj.weight', 'vision_model.encoder.layers.15.self_attn.k_proj.bias', 'vision_model.encoder.layers.20.self_attn.k_proj.weight', 'vision_model.encoder.layers.0.self_attn.q_proj.weight', 

In [ ]:
#Download the .ckpt file and use in your favorite Stable Diffusion repo!